In [1]:
import warnings
warnings.filterwarnings('ignore')
import torch
from torchvision import transforms, models, datasets
import numpy as np
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import os
from tqdm.auto import tqdm
from torch.utils.data import DataLoader, Dataset
from torch.nn import functional as F
import cv2
import subprocess
from collections import OrderedDict
import timm
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.metrics import roc_auc_score, auc
from sklearn.metrics import precision_score,recall_score, f1_score
import glob
import albumentations as A

In [2]:
import cv2

In [30]:
labels_csv = {'train': "/storage/home/akansh12/Vin-ChestXR-Abnormality-detection/Notebooks/weekly_supervised/new_image_labels_train.csv",
             'test': "/scratch/scratch6/akansh12/DeepEXrays/physionet.org/files/vindr-cxr/1.0.0/annotations/image_labels_test.csv"
             }
labels_csv_bb = {'train': "/scratch/scratch6/akansh12/DeepEXrays/physionet.org/files/vindr-cxr/1.0.0/annotations/annotations_train.csv",
             'test': "/scratch/scratch6/akansh12/DeepEXrays/physionet.org/files/vindr-cxr/1.0.0/annotations/annotations_test.csv"
             }
data_dir = {'train': "/scratch/scratch6/akansh12/DeepEXrays/data/data_256/train/",
           'test': "/scratch/scratch6/akansh12/DeepEXrays/data/data_256/test/"}

In [31]:
global_labels = ['COPD', 'Lung tumor', 'Pneumonia', 'Tuberculosis', 'Other diseases', 'No finding']

local_labels = ['Aortic enlargement', 'Atelectasis',
       'Calcification', 'Cardiomegaly', 'Clavicle fracture', 'Consolidation',
       'Edema', 'Emphysema', 'Enlarged PA', 'ILD', 'Infiltration',
       'Lung Opacity', 'Lung cavity', 'Lung cyst', 'Mediastinal shift',
       'Nodule/Mass', 'Pleural effusion', 'Pleural thickening', 'Pneumothorax',
       'Pulmonary fibrosis', 'Rib fracture', 'Other lesion', 'No finding']

In [32]:
train_label = pd.read_csv(labels_csv['train'])
test_label = pd.read_csv(labels_csv['test'])
train_label_bb = pd.read_csv(labels_csv_bb['train'])
test_label_bb = pd.read_csv(labels_csv_bb['test'])

In [60]:
class Vin_big_dataset(Dataset):
    def __init__(self, image_loc, label_loc, transforms, data_type, selec_radio, radio_id = None, label_type = None):
        if label_type == 'global':
            global_labels = ['COPD', 'Lung tumor', 'Pneumonia', 'Tuberculosis', 'Other diseases', 'No finding']
        if label_type == 'local':
            global_labels = ['Aortic enlargement', 'Atelectasis',
       'Calcification', 'Cardiomegaly', 'Clavicle fracture', 'Consolidation',
       'Edema', 'Emphysema', 'Enlarged PA', 'ILD', 'Infiltration',
       'Lung Opacity', 'Lung cavity', 'Lung cyst', 'Mediastinal shift',
       'Nodule/Mass', 'Pleural effusion', 'Pleural thickening', 'Pneumothorax',
       'Pulmonary fibrosis', 'Rib fracture', 'Other lesion', 'No finding']
        
        if data_type == 'train':
            label_df = pd.read_csv(label_loc)
            if selec_radio == 'rand_one':
                label_df['labels'] = label_df['image_id']
                label_df.set_index("labels", inplace = True)
                filenames = np.unique(label_df.index.values).tolist()
                self.full_filenames = [os.path.join(image_loc, i +'.png') for i in filenames]
                self.labels = []
                for i in (filenames):
                    self.labels.append(label_df[global_labels].loc[i].values.tolist()[np.random.choice([0,1,2])])
                self.labels = torch.tensor(self.labels)
            if selec_radio == 'agree_two':
                label_df['labels'] = label_df['image_id']
                label_df.set_index("labels", inplace = True)
                filenames_temp = np.unique(label_df.index.values).tolist()
                self.labels = []
                filenames = []
                for i in filenames_temp:
                    a,b = np.unique(label_df.loc[i][global_labels].values, axis = 0, return_counts=True)
                    if b[0] >= 2:
                        filenames.append(i)
                        self.labels.append(a[0])
                self.labels = torch.tensor(self.labels)
                self.full_filenames = [os.path.join(image_loc, i +'.png') for i in filenames]
            if selec_radio == 'agree_three':
                label_df['labels'] = label_df['image_id']
                label_df.set_index("labels", inplace = True)
                filenames_temp = np.unique(label_df.index.values).tolist()
                self.labels = []
                filenames = []
                for i in filenames_temp:
                    a,b = np.unique(label_df.loc[i][global_labels].values, axis = 0, return_counts=True)
                    if b[0] == 3:
                        filenames.append(i)
                        self.labels.append(a[0])
                self.labels = torch.tensor(self.labels)
                self.full_filenames = [os.path.join(image_loc, i +'.png') for i in filenames]
            if selec_radio == 'radio_per_epoch':
                label_df['labels'] = label_df['image_id']
                label_df.set_index("labels", inplace = True)
                filenames = np.unique(label_df.index.values).tolist()
                self.labels = []
                for i in filenames:
                    self.labels.append(label_df.loc[i][global_labels].values[radio_id].tolist())
                self.labels = torch.tensor(self.labels)
                self.full_filenames = [os.path.join(image_loc, i +'.png') for i in filenames]
            if selec_radio == 'all': 
#                 label_df['labels'] = label_df['image_id'] +'_'+ label_df['rad_id']
                label_df.set_index("image_id", inplace = True)
                filenames = label_df.index.values.tolist()
            
                self.full_filenames = [os.path.join(image_loc,i+'.png') for i in filenames]
                self.labels = []
                for i in tqdm(filenames):
                    self.labels.append(label_df[global_labels].loc[i].values.tolist())         
                self.labels = torch.tensor(self.labels)
                
        if data_type == 'test':                     
            filenames = os.listdir(image_loc)
            self.full_filenames = [os.path.join(image_loc, i) for i in filenames]
            label_df = pd.read_csv(label_loc)
            label_df.set_index("image_id", inplace = True)
            self.labels = [label_df[global_labels].loc[filename[:-4]].values for filename in filenames]
            
        self.transforms = transforms
        self.norm_transform = A.Normalize()
#         self.data_type = data_type
    def __len__(self):
        return len(self.full_filenames)
    
    def __getitem__(self, idx):
        image = Image.open(self.full_filenames[idx])
        augmented = self.transforms(image=np.array(image))
        img = augmented['image']
        img = self.norm_transform(image=img)["image"]
        
        return torch.tensor(img), self.labels[idx]

In [61]:
train_transforms = A.Compose([
    A.HorizontalFlip(p=0.3),
    A.VerticalFlip(p = 0.2),
    A.RandomRotate90(p=0.1),
    A.Rotate((-30,30), p = 0.5),
    A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=10, p=0.3),
    A.CLAHE(p=0.5),
    A.RandomBrightnessContrast(p=5)
])



test_transforms = A.Compose([
    A.Rotate(0, p = 1)
        ])



In [62]:
# data_transforms = { 
#     "train": transforms.Compose([
#         transforms.RandomHorizontalFlip(p = 0.5), 
#         transforms.RandomPerspective(distortion_scale=0.3),
#         transforms.RandomRotation((-30,30)),
#         transforms.ToTensor(),
#         transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])
#     ]),
    
#     "test": transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])        
#     ])
    
# }

In [63]:
train_data = Vin_big_dataset(image_loc = data_dir['train'],
                          label_loc = labels_csv['train'],
                          transforms = train_transforms,
                          data_type = 'train', selec_radio = 'all', label_type = 'local')

test_data = Vin_big_dataset(image_loc = data_dir['test'],
                          label_loc = labels_csv['test'],
                          transforms = test_transforms,
                          data_type = 'test', selec_radio = None, label_type = 'local')

trainloader = DataLoader(train_data,batch_size = 4,shuffle = False)
testloader = DataLoader(test_data,batch_size = 8,shuffle = False)


  0%|          | 0/15000 [00:00<?, ?it/s]

In [64]:
for x, y in trainloader:
    print(x.shape)
    print(y)
    break

torch.Size([4, 256, 256, 3])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
        [0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]])


In [65]:
model = timm.create_model('efficientnet_b6', pretrained=False)
model.load_state_dict(torch.load("/storage/home/akansh12/Vin-ChestXR-Abnormality-detection/model/tf_efficientnet_b6_aa-80ba17e4.pth"))

<All keys matched successfully>

In [66]:
model.classifier = nn.Sequential(OrderedDict([
    ('fcl1', nn.Linear(2304,23)),
    ('out', nn.Sigmoid()),
]))

In [67]:
def train_one_epoch(model, optimizer, lr_scheduler,
                    dataloader, epoch, criterion, device):
    
    print("Start Train ...")
    model.train()

    losses_train = []
    model_train_result = []
    train_target = []


    for data, targets in tqdm(dataloader):
        data = data.permute(0,3,1,2).to(device).float()
        targets = targets.to(device).type(torch.float)


        outputs = model(data)
        model_train_result.extend(outputs.detach().cpu().numpy().tolist())
        train_target.extend(targets.cpu().numpy())


        loss = criterion(outputs, targets)

        losses_train.append(loss.item())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    F1_train = f1_score(train_target, (np.array(model_train_result) > 0.5), average=None)

        
    if lr_scheduler is not None:
        lr_scheduler.step()

    lr = lr_scheduler.get_last_lr()[0]
    print("Epoch [%d]" % (epoch),
          "Mean loss on train:", np.array(losses_train).mean(), 
          "F1 score:",np.array(F1_train),
          "Learning Rate:", lr)

    
    return np.array(losses_train).mean(), np.array(F1_train), lr


def val_epoch(model, dataloader, epoch, criterion, device):
    
    print("Start Validation ...")
    model.eval()
    
    model_val_result = []
    val_target = []
    losses_val = []

    with torch.no_grad():
        for data, targets in tqdm(dataloader):

            data = data.permute(0,3,1,2).to(device).float()
            targets = targets.to(device).type(torch.float)

            outputs = model(data)
            
            #loss
            loss = criterion(outputs, targets)
            losses_val.append(loss.item())

            
            model_val_result.extend(outputs.detach().cpu().numpy().tolist())
            val_target.extend(targets.cpu().numpy())
            
        F1_val = f1_score(val_target, (np.array(model_val_result) > 0.5), average=None)




        print("Epoch:  " + str(epoch) + " F1 valid Score:", np.array(F1_val), 
              "Mean Loss", np.array(losses_val).mean())
        
    return np.array(losses_val).mean(), np.array(F1_val)

In [68]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.cuda.is_available():
    model = model.cuda()
    model = nn.DataParallel(model)

In [69]:
for param in model.parameters():
    param.requires_grad = True
num_epochs = 50
optimizer = optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr = 0.1, steps_per_epoch=1, epochs=50)

In [70]:
criterion = nn.BCELoss(reduction= 'mean')

In [71]:
weights_dir = "/scratch/scratch6/akansh12/DeepEXrays/local_label/efficient-net/"

train_loss_history = []
val_loss_history = []
train_f1_history = []
val_f1_history = []
lr_history = []

for epoch in range(num_epochs):

#     train_loss, train_f1, lr = train_one_epoch(model, optimizer, lr_scheduler,trainloader, epoch, criterion, device = device)

    val_loss, val_f1 = val_epoch(model, testloader, epoch, criterion, device = device)


    # train history
    train_loss_history.append(train_loss)
    train_f1_history.append(train_f1)
    lr_history.append(lr)

    #val history
    val_loss_history.append(val_loss)
    val_f1_history.append(val_f1)

    # save best weights
    best_loss = min(val_loss_history)
    if val_loss <= best_loss:
        print('saving model')
        torch.save({'state_dict': model.state_dict()},
                    os.path.join(weights_dir, f"{val_loss:0.6f}_.pth"))



Start Validation ...


  0%|          | 0/375 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [58]:
best_loss

0.6605295968937377

In [59]:
val_loss_history

[0.6605295968937377]

In [72]:
x

tensor([[[[ 0.2967,  0.4328,  0.6531],
          [ 0.2967,  0.4328,  0.6531],
          [ 0.2967,  0.4328,  0.6531],
          ...,
          [ 0.4337,  0.5728,  0.7925],
          [ 1.2385,  1.3957,  1.6117],
          [ 0.7933,  0.9405,  1.1585]],

         [[ 0.2282,  0.3627,  0.5834],
          [ 0.2624,  0.3978,  0.6182],
          [ 0.2624,  0.3978,  0.6182],
          ...,
          [ 0.8447,  0.9930,  1.2108],
          [ 0.6734,  0.8179,  1.0365],
          [-0.1486, -0.0224,  0.1999]],

         [[ 0.1768,  0.3102,  0.5311],
          [ 0.2282,  0.3627,  0.5834],
          [ 0.2111,  0.3452,  0.5659],
          ...,
          [ 0.2967,  0.4328,  0.6531],
          [-0.2513, -0.1275,  0.0953],
          [-0.7308, -0.6176, -0.3927]],

         ...,

         [[ 1.6838,  1.8508,  2.0648],
          [ 1.6838,  1.8508,  2.0648],
          [ 1.6838,  1.8508,  2.0648],
          ...,
          [ 1.0159,  1.1681,  1.3851],
          [ 0.8789,  1.0280,  1.2457],
          [ 0.6221,  0